<a href="https://colab.research.google.com/github/AdamShelby278/Mini-Project/blob/main/Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bing-image-downloader

Collecting Images


In [2]:
cd /content

/content


In [3]:
!mkdir Mini_P

In [4]:
import bing_image_downloader
from  bing_image_downloader import downloader
downloader .download('casual shoes', limit=100, output_dir='Mini_p', adult_filter_off=True, force_replace=False, timeout=60)
downloader .download('formal shoes', limit=100, output_dir='Mini_p', adult_filter_off=True, force_replace=False, timeout=60)



[!!]Indexing page: 1

[%] Indexed 12 Images on Page 1.


[%] Downloading Image #1 from http://assets0.chictopia.com/photos/reyalfashion/1693765927/navy-vans-shoes-navy-h-m-shirt-beige-h-m-shorts-navy-paul-smith-sunglasses.jpg
[%] File Downloaded !

[%] Downloading Image #2 from http://bmodish.com/wp-content/uploads/2014/05/stripe-shirt-business-casual-bmodish.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://sapphirediaries.com/wp-content/uploads/2015/07/untitled-8402.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://glamradar.com/wp-content/uploads/2014/11/casual-clothes-and-platform-shoes.jpg
[!] Issue getting: https://glamradar.com/wp-content/uploads/2014/11/casual-clothes-and-platform-shoes.jpg
[!] Error:: HTTP Error 523: Origin Unreachable
[%] Downloading Image #4 from https://thumbs.dreamstime.com/x/girl-s-feet-converse-sneakers-4636261.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://glamradar.com/wp-content/uploads/2015/10/1.-skin

Preprocess Data

In [7]:
from skimage.transform import resize
from skimage.io import imread
import matplotlib.pyplot as plt
import os
import numpy as np

target = []
images = []     # matrix format
flat_data = []  # vector format

DATADIR = '/content/Mini_p'
CATEGORIES = ['casual shoes','formal shoes']
for category in CATEGORIES:
  class_num = CATEGORIES.index(category)
  path = os.path.join(DATADIR,category)
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_num)

target = np.array(target)
images = np.array(images)
flat_data = np.array(flat_data)

Dataframe

In [8]:
import pandas as pd

df = pd.DataFrame(flat_data)
df['Target'] = target
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,67461,67462,67463,67464,67465,67466,67467,67468,67469,67470,67471,67472,67473,67474,67475,67476,67477,67478,67479,67480,67481,67482,67483,67484,67485,67486,67487,67488,67489,67490,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,0.996078,...,0.582353,0.364706,0.396078,0.575294,0.333333,0.356471,0.584314,0.313725,0.325490,0.586235,0.305882,0.317647,0.580392,0.301961,0.313725,0.576471,0.298039,0.309804,0.588235,0.310078,0.325490,0.603922,0.337255,0.360431,0.600000,0.345098,0.372549,0.561176,0.315176,0.337647,0.533333,0.282353,0.294118,0.537255,0.274510,0.282353,0.541176,0.274510,0.282353,0
1,0.990196,0.996078,0.933333,0.990196,0.992157,0.931373,0.992157,0.996078,0.933333,0.996078,0.998039,0.937255,0.996078,1.000000,0.937255,0.996078,1.000000,0.937255,0.996078,1.000000,0.937255,0.992157,0.996078,0.933333,0.992157,0.996078,0.927451,0.994118,0.998039,0.933333,0.996078,1.000000,0.937255,0.996078,1.000000,0.943137,0.996078,1.000000,0.945098,0.996078,...,0.241176,0.200000,0.254902,0.227451,0.182353,0.233333,0.239216,0.184314,0.231373,0.264706,0.200000,0.245098,0.284314,0.215686,0.260784,0.305882,0.231373,0.276471,0.321569,0.243137,0.288235,0.323529,0.239216,0.274510,0.319608,0.231373,0.266667,0.323529,0.233333,0.266667,0.317647,0.235294,0.264706,0.321569,0.227451,0.262745,0.327451,0.227451,0.264706,0
2,0.507843,0.323529,0.200000,0.442137,0.283314,0.160784,0.486275,0.317647,0.207843,0.453902,0.307843,0.200961,0.421569,0.254902,0.167627,0.356824,0.218608,0.117647,0.445059,0.297039,0.164706,0.513725,0.376471,0.264706,0.467667,0.365667,0.260745,0.620490,0.654745,0.583157,0.972510,0.982353,0.979392,0.986275,0.990196,0.989235,0.986275,0.988235,0.988196,0.941176,...,0.988235,0.886275,0.734333,0.976471,0.849020,0.698078,0.950059,0.819686,0.662824,0.951980,0.812804,0.667706,0.959824,0.828490,0.682392,0.952980,0.822608,0.678471,0.956902,0.819725,0.676549,0.972588,0.847176,0.699118,0.981392,0.899039,0.765706,0.966667,0.893157,0.747059,0.982353,0.945137,0.813765,0.986275,0.964745,0.851059,0.984314,0.969627,0.858863,0
3,0.990196,0.996078,0.933333,0.990196,0.992157,0.931373,0.992157,0.996078,0.933333,0.996078,0.998039,0.937255,0.996078,1.000000,0.937255,0.996078,1.000000,0.937255,0.996078,1.000000,0.937255,0.992157,0.996078,0.933333,0.992157,0.996078,0.927451,0.994118,0.998039,0.933333,0.996078,1.000000,0.937255,0.996078,1.000000,0.943137,0.996078,1.000000,0.945098,0.996078,...,0.241176,0.200000,0.254902,0.227451,0.182353,0.233333,0.239216,0.184314,0.231373,0.264706,0.200000,0.245098,0.284314,0.215686,0.260784,0.305882,0.231373,0.276471,0.321569,0.243137,0.288235,0.323529,0.239216,0.274510,0.319608,0.231373,0.266667,0.323529,0.233333,0.266667,0.317647,0.235294,0.264706,0.321569,0.227451,0.262745,0.327451,0.227451,0.264706,0
4,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.133333,0.031373,0.035294,0.137255,...,0.517647,0.462745,0.396078,0.500000,0.445098,0.378431,0.490196,0.431373,0.366667,0.443137,0.382353,0.321569,0.501961,0.458824,0.386275,0.441176,0.384314,0.317647,0.507843,0.468627,0.398039,0.484314,0.431373,0.362745,0.390196,0.333333,0.252941,0.405882,0.349020,0.268627,0.413725,0.350980,0.284314,0.498039,0.439216,0.368627,0.492157,0.437255,0.366667,0
...,...,...,...,...,...,...,...,...,...,...,...,.

Building Model

In [10]:
x = df.iloc[:,0:67500]
y = df.iloc[:,67500]

Spliting Data

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state = 0)

Using GridSearch CV to find best Parameters for K-Nearest Neighbours

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
z = np.arange(1,47)
param = {'n_neighbors':z}
model = KNeighborsClassifier()
model_grid = GridSearchCV(model,param)
model_grid.fit(x,y)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [13]:
model_grid.best_params_

{'n_neighbors': 1}

Fit and Transform the training and testing data

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
model = KNeighborsClassifier(n_neighbors=1,metric='euclidean')
model.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [16]:
y_pred = model.predict(x_test)

Evaluation of model

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.98

In [ ]:
#Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state = 1)
rf_model.fit(x_train,y_train)

y_pred2 = rf_model.predict(x_test)


In [19]:
for i in range(len(y_pred2)):
  y_pred2[i] = round(y_pred2[i])

Accuracy Test 

In [44]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_pred,y_test)

0.98

Absolute mean of all Errors

In [47]:
from sklearn.metrics import mean_absolute_error
rf_test_mae = mean_absolute_error(y_pred, y_test)
rf_test_mae

0.02

Confusion Matrix

In [27]:
confusion_matrix(y_pred,y_test)

array([[23,  1],
       [ 0, 26]])

Testing New Image

In [ ]:
Flat_data = []
url = input('Enter your URL')
img_resized = resize(img_array,(150,150,3))
flat_data.append(img_resized.flatten())
falt_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = CATEGORIES[y_out[0]]
print(f' PREDICTED OUTPUT: {y_out}')